Universidad del Valle de Guatemala
Luis Esturban 17256
17/04/2020
HDT2

Se importan las librerias y modulos necesarios

In [1]:
import numpy as np
import pandas as pd
import pickle
from functools import reduce
from scipy import optimize as op
import Modulos as md

Se cargan los datos y se crean las particiones para train y test

In [2]:
dTrain = pd.read_csv('fashion-mnist_train.csv')
dTest = pd.read_csv('fashion-mnist_test.csv')
df = [dTrain, dTest]
data = pd.concat(df)
data = data.sample(frac = 1)
nData = len(data)
train = data.iloc[:int(nData * 0.6), :]
test = data.iloc[int(nData * 0.6):int(nData * 0.8), :]
crossV = data.iloc[int(nData * 0.8):, :]

Se separa x e y de train

In [3]:
X = train.iloc[:, 1:] / 1000.0 
m, n = X.shape
y = np.asarray(train.iloc[:, 0])
y = y.reshape(m, 1)
Y = (y == np.array(range(10))).astype(int)

Se crea la arquitectura de la red y se procede a crear el modelo

In [4]:
NETWORK_ARCHITECTURE = np.array([n,130,10])
with (open("iNeurons", "rb")) as openfile:
    while True:
        try:
            iTht = pickle.load(openfile)
        except EOFError:
            break
openfile.close()
tht = iTht
fTht = np.hstack((NETWORK_ARCHITECTURE[1:].reshape(len(NETWORK_ARCHITECTURE) - 1, 1),(NETWORK_ARCHITECTURE[:-1] + 1).reshape(len(NETWORK_ARCHITECTURE) - 1, 1)))

Se inicia el entrenamiento

In [5]:
print("Entrenando")
result = op.minimize(fun=md.cost,x0=tht,args=(fTht, X, Y),method='L-BFGS-B',jac=md.cbn,options={'disp': True, 'maxiter': 2000})
print("Entrenado")

Entrenando


C:\Users\lcest\Desktop\Lab IA 2\IA_HDT2\Modulos.py:18: RuntimeWarning: divide by zero encountered in log
  return -(Y * np.log(a[-1]) + (1 - Y) * np.log(1 - a[-1])).sum() / len(X)
C:\Users\lcest\Desktop\Lab IA 2\IA_HDT2\Modulos.py:18: RuntimeWarning: invalid value encountered in multiply
  return -(Y * np.log(a[-1]) + (1 - Y) * np.log(1 - a[-1])).sum() / len(X)


Entrenado


Se crea un diccionario y se guarda el resultado en otro modelo

In [6]:
dicc = {
  0: "T-shirt/top",
  1: "Trouser",
  2: "Pullover",
  3: "Dress",
  4: "Coat",
  5: "Sandal",
  6: "Shirt",
  7: "Sneaker",
  8: "Bag",
  9: "Ankle boot"
}
outfile = open("iNeuronsF", "wb")
pickle.dump(result.x, outfile)
outfile.close()

Se separa x e y de test

In [7]:
X = test.iloc[:, 1:] / 1000.0
m, n = X.shape
y = np.asarray(test.iloc[:, 0])
y = y.reshape(m, 1)
Y = (y == np.array(range(10))).astype(int)

se procede a trabajar con el modelo nuevo

In [8]:
with (open("iNeuronsF", "rb")) as openfile:
    while True:
        try:
            oTht = pickle.load(openfile)
        except EOFError:
            break
fTht = np.hstack((NETWORK_ARCHITECTURE[1:].reshape(len(NETWORK_ARCHITECTURE) - 1, 1),(NETWORK_ARCHITECTURE[:-1] + 1).reshape(len(NETWORK_ARCHITECTURE) - 1, 1)))
res = md.pdt(oTht, fTht, X, Y)

Se opera para obtener los valores de las predicciones

In [9]:
prd = []
vTrd = []
trd = 0
fls = 0
for row1 in res[0]:
    rst1 = np.where(row1 == np.amax(row1))
    prd.append(rst1[0])
for row2 in res[1]:
    rst2 = np.where(row2 == np.amax(row2))
    vTrd.append(rst2[0])
for a in range(len(X)):
    if prd[a].item(0) == vTrd[a].item(0):
        trd = trd + 1
    else:
        fls = fls + 1
print("Porcentaje: ", round(100 * trd/(trd + fls), 2), "%")

Porcentaje:  9.4 %


Se realiza Cross Validation

In [10]:
X = crossV.iloc[:, 1:] / 1000.0
m, n = X.shape
y = np.asarray(crossV.iloc[:, 0])
y = y.reshape(m, 1)
Y = (y == np.array(range(10))).astype(int)
with (open("iNeuronsF", "rb")) as openfile:
    while True:
        try:
            oTht = pickle.load(openfile)
        except EOFError:
            break
fTht = np.hstack((NETWORK_ARCHITECTURE[1:].reshape(len(NETWORK_ARCHITECTURE) - 1, 1),(NETWORK_ARCHITECTURE[:-1] + 1).reshape(len(NETWORK_ARCHITECTURE) - 1, 1)))
res = md.pdt(oTht, fTht, X, Y)

Se vulve a operar para obtener los valores de las predicciones

In [11]:
prd = []
vTrd = []
trd = 0
fls = 0
for row1 in res[0]:
    rst1 = np.where(row1 == np.amax(row1))
    prd.append(rst1[0])
for row2 in res[1]:
    rst2 = np.where(row2 == np.amax(row2))
    vTrd.append(rst2[0])
for a in range(len(X)):
    if prd[a].item(0) == vTrd[a].item(0):
        trd = trd + 1
    else:
        fls = fls + 1
print("Porcentaje: ", round(100 * trd/(trd + fls), 2), "%")

Porcentaje:  10.13 %
